# EDA on time-based features

In [0]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/youtube_playlog.csv"
ACCESS_KEY_ID = " " # cle du compte student
SECRET_ACCESS_KEY = " " # secret key du compte student

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", ACCESS_KEY_ID)
hadoop_conf.set("fs.s3a.secret.key", SECRET_ACCESS_KEY)
hadoop_conf.set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") 

INPUT_FILENAME = 's3://full-stack-bigdata-datasets/Big_Data/playlog_processed.parquet'

In [0]:
# load data
### BEGIN STRIP ###
df_raw = spark.read.load(INPUT_FILENAME)
df_raw.count()
### END STRIP ###

Out[15]: 25615885

In [0]:
# As a refresher
df_raw.limit(5).toPandas()

Out[16]:

,timestamp,user,song
0,1392388988,16,ba_ztQx87Ew
1,1392389603,101,KsMEBEcxzYA
2,1392393824,180,uyrtLz_ShL4
3,1392394627,193,rnQG04SkWD4
4,1392387742,15,e7jMGpzDBKI


In [0]:
# compute a new column `datetime`
#       that converts the timestamp to a datetime
#       drop the `timestamp` column
#       and order by `datetime`
#       save this as a new DataFrame `df`
#       show the first 5 rows of `df`
### BEGIN STRIP ###
from pyspark.sql import functions as F

df = (df_raw.withColumn('datetime', F.from_unixtime('timestamp')) \
      .drop('timestamp') \
      .orderBy('datetime') \
      .select('datetime', 'user', 'song'))
df.show(5)

### END STRIP ###

+-------------------+----+-----------+
 datetime|user| song|
+-------------------+----+-----------+
2014-02-14 14:18:53| 0|t1l8Z6gLPzo|
2014-02-14 14:18:57| 70|VJ6ofd0pB_c|
2014-02-14 14:18:57| 22|Q24VZL8wpOM|
2014-02-14 14:18:58| 1|t1l8Z6gLPzo|
2014-02-14 14:18:59| 40|d12dJbE_xxw|
+-------------------+----+-----------+
only showing top 5 rows

Now that we have a datetime column, we can compute new columns, namely:
- [year](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=date#pyspark.sql.functions.year)
- [month](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=date#pyspark.sql.functions.month)
- [dayofmonth](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=date#pyspark.sql.functions.dayofmonth)
- [dayofweek](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=date#pyspark.sql.functions.dayofweek)
- [dayofyear](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=date#pyspark.sql.functions.dayofyear)
- [weekofyear](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=date#pyspark.sql.functions.weekofyear)

We will put the resulting DataFrame in a variable called `df_enriched`.

In [0]:
### BEGIN STRIP ###
from functools import reduce

funcs = [F.year, F.month, F.dayofmonth, F.dayofweek, F.dayofyear, F.weekofyear]

df_enriched = reduce(
  lambda memo_df, f: memo_df.withColumn(f.__name__, f('datetime')),
  funcs, df)

# alternatively
# df_enriched = df.select('*', *(f('datetime').alias(f.__name__) for f in funcs))

df_enriched.show()
### END STRIP ###

+-------------------+----+-----------+----+-----+----------+---------+---------+----------+
 datetime|user| song|year|month|dayofmonth|dayofweek|dayofyear|weekofyear|
+-------------------+----+-----------+----+-----+----------+---------+---------+----------+
2014-02-14 14:18:53| 0|t1l8Z6gLPzo|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:57| 70|VJ6ofd0pB_c|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:57| 22|Q24VZL8wpOM|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:58| 1|t1l8Z6gLPzo|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:59| 40|d12dJbE_xxw|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:04| 73|OrxxHlRs85U|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:12| 38|guKoNCQFAFk|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:16| 2|t1l8Z6gLPzo|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:16| 43|WGEYWgSGx9A|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:21| 3|we5gzZq5Avg|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:21| 36|6dQ__nEY3ts|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:26| 119|idHAwYjSNCk|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:26| 4|we5gzZq5Avg|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:26| 5|we5gzZq5Avg|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:27| 6|49esza4eiK4|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:31| 45|t6bjqdll7DI|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:32| 95|F5pzQviQZwE|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:34| 6|49esza4eiK4|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:36| 6|VKzWLUQizz8|2014| 2| 14| 6| 45| 7|
2014-02-14 14:19:38| 52|VwvphO-Oyu4|2014| 2| 14| 6| 45| 7|
+-------------------+----+-----------+----+-----+----------+---------+---------+----------+
only showing top 20 rows

In [0]:
# print out the schema of the new dataframe
### BEGIN STRIP ###
df_enriched.printSchema()
### END STRIP ###

root
-- datetime: string (nullable = true)
-- user: integer (nullable = true)
-- song: string (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- dayofmonth: integer (nullable = true)
-- dayofweek: integer (nullable = true)
-- dayofyear: integer (nullable = true)
-- weekofyear: integer (nullable = true)

In [0]:
# Plot average of monthly counts
### BEGIN STRIP ###
display(df_enriched.groupBy('month').count().orderBy('month'))
### END STRIP ###

month,count
1,2003539
2,2147710
3,2518750
4,2265007
5,2366518
6,2086337
7,1953630
8,1760367
9,1966203
10,2386054


In [0]:
# clean
### BEGIN STRIP ###
def count_by_period(col_name, df):
    return df.groupBy(col_name).count().orderBy(col_name)

from functools import partial
### END STRIP ###

In [0]:
# bar plot by year
### BEGIN STRIP ###
display(df_enriched.transform(partial(count_by_period, 'year')))
### END STRIP ###

year,count
2014,8041498
2015,8143052
2016,6017753
2017,2468620
2018,819927
2019,125035


In [0]:
# bar plot by month
### BEGIN STRIP ###
display(df_enriched.transform(partial(count_by_period, 'month')))
### END STRIP ###

month,count
1,2003539
2,2147710
3,2518750
4,2265007
5,2366518
6,2086337
7,1953630
8,1760367
9,1966203
10,2386054


In [0]:
# bar plot by weekofyear
### BEGIN STRIP ###
display(df_enriched.transform(partial(count_by_period, 'weekofyear')))
### END STRIP ###

weekofyear,count
1,434786
2,424200
3,466600
4,496326
5,473586
6,477376
7,505821
8,581735
9,600993
10,568131


In [0]:
# bar plot by dayofmonth
### BEGIN STRIP ###
display(df_enriched.transform(partial(count_by_period, 'dayofmonth')))
### END STRIP ###

dayofmonth,count
1,830349
2,858332
3,820602
4,848713
5,850318
6,871429
7,835162
8,832550
9,842589
10,845356


In [0]:
# bar plot by dayofyear
### BEGIN STRIP ###
display(df_enriched.transform(partial(count_by_period, 'dayofyear')))
### END STRIP ###

dayofyear,count
1,41434
2,48587
3,45183
4,60572
5,53649
6,61874
7,60580
8,62790
9,79198
10,55000
